In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# PreCode

In [2]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
# l

recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

data/ProcData/TensorFolder/Task2YearXXX


In [3]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
    print(tensor_folder)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

data/ProcData/TensorFolder/Task2YearXXX/P@age-AgeNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX/P@basicInfo-basicInfoDftGrn
data/ProcData/TensorFolder/Task2YearXXX/EC@BasicInfo-BasicDftGrn
data/ProcData/TensorFolder/Task2YearXXX/EC@DT_min-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/A1C@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/A1C@V-A1CNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX/Diag@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/Diag@Value-DiagDftGrn
data/ProcData/TensorFolder/Task2YearXXX/PNSectSent@Sentence-Tk@TknzLLMGrn
B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [4]:
full_recfldgrn_list = [k for k in batch_rfg]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [5]:
import torch 

RECFLD_TO_TENSOR = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
   
    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
    
    RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    

In [6]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_E_subunit_name = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 




df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_E_subunit_name)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
1,E,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'Male': 1...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
2,E,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'X': 1, '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
3,E,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
4,E,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
5,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
6,E,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
7,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
8,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': BertTokenizerFast(name_or...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [7]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


# Get Input Tensors

In [8]:

for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


In [9]:
max_layer = max([len(i.split('-')) for i in RECFLD_TO_EMBEDTESNOR])
max_layer

7

# Get df DataFlow

In [10]:
# fieldnn.utils.datanamefn.py

def get_curfld_recinfo(curfld):
    '''
        a helper function that get the rec information from the current recfldgrn name.
        the input is: prefix_rec@fld_grn
        we want to check whether @ is here. 
        If there is a @ in the current level, we want to put the rec as the potential merger target. 
    '''
    if '@' in curfld:
        fld_list = curfld.split('@')
        return '@'.join(fld_list[:-1]), fld_list[-1]
    else:
        return None, curfld

In [11]:
# fieldnn.dataflowfn.reprflowfn.py

import numpy as np
import pandas as pd


def get_Repr_dataflow_table(full_recfldgrn_list):
    '''
        input: a list of full_recfldgrn
        output: the dataframe that shows the data flow from the grain-embedding-tensor to final feature vector.
    '''
    max_layer = max([len(i.split('-')) for i in full_recfldgrn_list])
    L = []
    for full_recfldgrn in full_recfldgrn_list:
        d = {}
        recfldgrn = '-'.join(full_recfldgrn.split('-')[-2:])
        current_layer_id = len(full_recfldgrn.split('-'))
        d['recfldgrn'] = recfldgrn
        for layer_id in range(max_layer, 1, -1):
            if layer_id > current_layer_id:
                d[layer_id] = np.nan
            elif layer_id == current_layer_id:
                d[layer_id] = full_recfldgrn
            else:
                d[layer_id] = np.nan
        L.append(d)
    df_dataflow = pd.DataFrame(L).set_index('recfldgrn')
    return df_dataflow


In [12]:
def update_df_Repr_dataflow(df, style = 'Reducer&Merger'):
    
    '''
        This function try to auto-fill the data flow from each field to the end field 'B-P'.
        There are two styles:
            style 'Reducer&Merger': Merging the tensors along the data flow.
            style 'ReducerOnly': Don't merge the tensors until the end of the models. 
    
    '''
    
    
    df_dataflow = df.copy()

    for layer_idx in df_dataflow.columns:
        
        # (1) first deal with the fld in each layer
        s = df_dataflow[layer_idx]
        current_fullrfg_list = s[-s.isna()].to_list()
        fld_with_at = [i for i in current_fullrfg_list if '@' in i.split('-')[-1]]
        fld_with_at_to_keep = []
        fld_with_at_to_check = [i for i in fld_with_at if i not in fld_with_at_to_keep]
        
        while len(fld_with_at_to_check) > 0:
            for index in df_dataflow.index:
                full_recfldgrn = df_dataflow.loc[index, layer_idx]
                if pd.isna(full_recfldgrn): continue
                if full_recfldgrn == 'ToFill': continue
                
                if '@' in full_recfldgrn.split('-')[-1]:
                    
                    prefix = '-'.join(full_recfldgrn.split('-')[:-1])
                    curfld = full_recfldgrn.split('-')[-1] # PNSectSent@Sentence@Tk
                    headfld, tailfld = get_curfld_recinfo(curfld) # head: PNSectSent@Sentence, tail: Tk
                    
                    full_recfldgrn_new = full_recfldgrn.replace('@' + tailfld, '')
                    same_rfg_new = [i for i in current_fullrfg_list if full_recfldgrn_new in i]
                    
                    # print(f'for Layer {layer_idx}')
                    # print(full_recfldgrn, '<--- full_recfldgrn')
                    # print(full_recfldgrn_new, '<--- full_recfldgrn_new')
                    # print(same_rfg_new, '<--- same_rfg_new')
                    
                    if len(same_rfg_new) == 1:
                        # print(full_recfldgrn)
                        df_dataflow.loc[index, layer_idx] = full_recfldgrn_new
                        ############# need to think about this
                        s = df_dataflow[layer_idx]
                        current_fullrfg_list = s[-s.isna()].to_list()
                        #############
                        

                    elif len(same_rfg_new) > 1:
                        
                        if style == 'Reducer&Merger':
                            fld_with_at_to_keep.extend(same_rfg_new)
                            
                            # To update a new Column
                            tailfld_merged = '&'.join([i.replace(full_recfldgrn_new +'@', '') for i in same_rfg_new])
                            new_index = '(Merge)' + headfld + '@' + tailfld_merged#
                            if new_index in df_dataflow.index: continue

                            selected_index_list = [df_dataflow[df_dataflow[layer_idx] == i].index[0] for i in same_rfg_new]
                            # print(selected_index_list)

                            l = list(df_dataflow.index)
                            # print(l, '<--- l original columns')
                            loc = max([l.index(i) for i in selected_index_list])
                            l.insert(loc + 1, new_index)
                            # print(l, '<--- l with new index columns')

                            df_dataflow.loc[new_index, layer_idx] = full_recfldgrn_new
                            df_dataflow = df_dataflow.reindex(l)
                            # print(list(df_dataflow.index), '<--- df_dataflow with new index columns')

                            ############# need to think about this
                            # s = df_dataflow[layer_idx]
                            # current_fullrfg_list = s[-s.isna()].to_list()
                            #############
                            fld_with_at_to_keep.append(full_recfldgrn_new)
                            # print(fld_with_at_to_keep, '<----------------'
                            
                        elif style == 'ReducerOnly':
                            fld_with_at_to_keep.extend(same_rfg_new)
                            
                        else:
                            raise ValueError('Wrong style, only for Reducer&Merger and ReducerOnly')
                            
            
            # (2) update conditions
            s = df_dataflow[layer_idx]
            current_fullrfg_list = s[-s.isna()].to_list()
            fld_with_at = [i for i in current_fullrfg_list if '@' in i.split('-')[-1]]
            fld_with_at_to_check = [i for i in fld_with_at if i not in fld_with_at_to_keep]
            
        if layer_idx == df_dataflow.columns[-1]: continue
        # then update the next layer information
        for index in df_dataflow.index:
            last = df_dataflow.loc[index, layer_idx]
            if pd.isna(last): continue
            if '@' in last.split('-')[-1] and style == 'Reducer&Merger': continue
            full_recfldgrn_next = '-'.join(last.split('-')[:-1]) + '@' + last.split('-')[- 1]
            df_dataflow.loc[index, layer_idx - 1] = full_recfldgrn_next
            
            
    if style == 'ReducerOnly':
        df_dataflow.loc['(Merge)P@All', 2] = 'B-P' 
        
    return df_dataflow

In [13]:
def update_df_Repr_dataflow_completename(df):
    df_dataflow = df.copy()
    L = []
    for recfldgrn, row in df_dataflow.iterrows():
        # print(recfldgrn)
        new_row = {}
        new_row['recfldgrn'] = recfldgrn
        full_recfldgrn = [i for i in row.values if not pd.isna(i)][0]
        curlayer_idx = len(full_recfldgrn.split('-'))
        for layer_idx, full_recfldgrn in row.items():
            if curlayer_idx < layer_idx:
                new_row[layer_idx] = np.nan
            elif curlayer_idx == layer_idx:
                new_row[layer_idx] = full_recfldgrn
            elif curlayer_idx > layer_idx:
                last = new_row[layer_idx+1]
                output_fullname = '-'.join(last.split('-')[:-1]) + '@' + last.split('-')[- 1]
                new_row[layer_idx] = output_fullname
            else:
                raise ValueError('wrong information')
        L.append(new_row)
    df_dataflow_filled = pd.DataFrame(L).set_index('recfldgrn')
    return df_dataflow_filled

In [14]:
full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [15]:
df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
df_dataflow

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,NaN
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,NaN
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,NaN,NaN
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,NaN,NaN
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,NaN,NaN,NaN
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,NaN,NaN,NaN
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,NaN,NaN,NaN
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,NaN,NaN,NaN
PNSectSent@Sentence-Tk@TknzLLM,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,NaN,NaN,NaN,NaN,NaN


In [16]:
df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'ReducerOnly')
df_dataflow_new

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,B-P@age
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,B-P@basicInfo
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,B-P-EC@BasicInfo,B-P@EC@BasicInfo
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,B-P-EC@DT_min,B-P@EC@DT_min
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,B-P-EC-A1C@DT,B-P-EC@A1C@DT,B-P@EC@A1C@DT
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,B-P-EC-A1C@V,B-P-EC@A1C@V,B-P@EC@A1C@V
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,B-P-EC-Diag@DT,B-P-EC@Diag@DT,B-P@EC@Diag@DT
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,B-P-EC-Diag@Value,B-P-EC@Diag@Value,B-P@EC@Diag@Value
PNSectSent@Sentence-Tk@TknzLLM,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk,B-P-EC-PN-PNSect-PNSectSent,B-P-EC-PN-PNSect,B-P-EC-PN,B-P-EC@PN,B-P@EC@PN


In [17]:
df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
df_dataflow_new

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,B-P@age
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,B-P@basicInfo
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,B-P-EC@BasicInfo,NaN
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,B-P-EC@DT_min,NaN
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,B-P-EC-A1C@DT,NaN,NaN
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,B-P-EC-A1C@V,NaN,NaN
(Merge)A1C@DT&V,NaN,NaN,NaN,B-P-EC-A1C,B-P-EC@A1C,NaN
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,B-P-EC-Diag@DT,NaN,NaN
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,B-P-EC-Diag@Value,NaN,NaN


# Get df SubUnit

In [18]:
df_dataflow = df_dataflow_new.copy()

In [19]:
layeridx_list = list(df_dataflow.columns)
layeridx_list

[7, 6, 5, 4, 3, 2]

In [20]:
for idx in range(len(layeridx_list) - 1):
    A_layerid = layeridx_list[idx]
    B_layerid = layeridx_list[idx + 1]
    print(A_layerid, B_layerid)

7 6
6 5
5 4
4 3
3 2


In [21]:
for idx in range(len(layeridx_list) - 1):
    A_layerid = layeridx_list[idx]
    B_layerid = layeridx_list[idx + 1]
    # print(A_layerid, B_layerid)
    
    
    A_tensors = df_dataflow[A_layerid]
    A_tensors = A_tensors[-A_tensors.isna()].to_dict()
    
    B_tensors = df_dataflow[B_layerid]
    B_tensors = B_tensors[-B_tensors.isna()].to_dict()
    
    
    print(f'\nFrom Layer {A_layerid} to {B_layerid}:')
    
    # from A tensor to B tensor, we have the Reducer NNs.
    # also notice that some 
    print('A', A_tensors)

    
    # from B tensor: potential there are some Merger NNs. 
    print('B', B_tensors)


From Layer 7 to 6:
A {'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk'}
B {'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect-PNSectSent'}

From Layer 6 to 5:
A {'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect-PNSectSent'}
B {'A1C@DT-DTDft': 'B-P-EC-A1C@DT-DTDft', 'A1C@V-A1CNumeDft': 'B-P-EC-A1C@V-A1CNumeDft', 'Diag@DT-DTDft': 'B-P-EC-Diag@DT-DTDft', 'Diag@Value-DiagDft': 'B-P-EC-Diag@Value-DiagDft', 'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect'}

From Layer 5 to 4:
A {'A1C@DT-DTDft': 'B-P-EC-A1C@DT-DTDft', 'A1C@V-A1CNumeDft': 'B-P-EC-A1C@V-A1CNumeDft', 'Diag@DT-DTDft': 'B-P-EC-Diag@DT-DTDft', 'Diag@Value-DiagDft': 'B-P-EC-Diag@Value-DiagDft', 'PNSectSent@Sentence-Tk@TknzLLM': 'B-P-EC-PN-PNSect'}
B {'EC@BasicInfo-BasicDft': 'B-P-EC@BasicInfo-BasicDft', 'EC@DT_min-DTDft': 'B-P-EC@DT_min-DTDft', 'A1C@DT-DTDft': 'B-P-EC-A1C@DT', 'A1C@V-A1CNumeDft': 'B-P-EC-A1C@V', '(Merge)A1C@DT&V': 'B-P-EC-A1C', 'Diag@DT-DTDft': 'B-P-EC-Diag@DT', 'Diag@Value-DiagDf

In [22]:

def get_Repr_SubUnit_List(df_dataflow, 
                          default_R_subunit_name = 'RL', 
                          default_MR_subunit_name = 'MRL', # or 'MLRL'
                         ):
    layeridx_list = list(df_dataflow.columns)
    
    SubUnit_List = []
    for idx in range(len(layeridx_list) - 1):
        A_layerid = layeridx_list[idx]
        B_layerid = layeridx_list[idx + 1]
        # print(A_layerid, B_layerid)

        A_tensors = df_dataflow[A_layerid]
        A_tensors = A_tensors[-A_tensors.isna()].to_dict()

        B_tensors = df_dataflow[B_layerid]
        B_tensors = B_tensors[-B_tensors.isna()].to_dict()

        # print(f'\nFrom Layer {A_layerid} to {B_layerid}:')

        # from A tensor to B tensor, we have the Reducer NNs.
        # also notice that some 

        # print('A', A_tensors)
        for k in A_tensors:
            if k not in B_tensors: continue

            input_name = A_tensors[k]
            output_name = B_tensors[k]

            if pd.isna(output_name) == True: continue # pass it. 

            d = {}
            d['SubUnitName'] = default_R_subunit_name
            d['input_names'] = [input_name]
            d['output_name'] = output_name
            
            d['input_layerid'] = A_layerid
            d['output_layerid'] = B_layerid
        
        
            SubUnit_List.append(d)


        # from B tensor: potential there are some Merger NNs. 
        # print('B', B_tensors)
        # check whether these is a '(Merger') in the key
        merger_tensors = [v for k, v in B_tensors.items() if '(Merge)' in k and '@' not in v]
        # print('B-merger_tensors', merger_tensors)

        for output_tensor in merger_tensors:
            input_tensors = [i for k, i in B_tensors.items() if output_tensor + '@' in i]
            # print(output_tensor, ':', input_tensors)

            d = {}
            d['SubUnitName'] = default_MR_subunit_name
            d['input_names'] = input_tensors
            d['output_name'] = output_tensor
            
            d['input_layerid'] = B_layerid
            d['output_layerid'] = B_layerid
        
            SubUnit_List.append(d)
            
    df_SubUnit = pd.DataFrame(SubUnit_List)
    return df_SubUnit




In [23]:
default_R_subunit_name = 'R'
default_MR_subunit_name = 'MR'

df_SubUnit = get_Repr_SubUnit_List(df_dataflow, default_R_subunit_name, default_MR_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3
